# Dataset Statistics

In [1]:
import os 
import json 
from collections import defaultdict
import pandas as pd
from pathlib import Path
import string
from inflection import pluralize
from util_wordnet import get_sister_terms

In [17]:
def read_bert_vocab(bert_vocab_path = 'data/bert-large-uncased-vocab.txt'):
    
    
    vocab = set()
    with open(bert_vocab_path, 'r') as fin: 
        lines = fin.readlines()
        for line in lines: 
            line = line.strip()
            vocab.add(line)
    return vocab        

bert_vocab= read_bert_vocab(bert_vocab_path = '../../data/bert-large-uncased-vocab.txt')

In [4]:
dataset_to_respath={
    "hypernymsuite-BLESS": "data/hypernymsuite/BLESS/IsA.jsonl",
    "lm_diagnostic_extended-singular": "data/lm_diagnostic_extended/singular/IsA.jsonl",
    "clsb-singular": "data/clsb/singular/IsA.jsonl",
    "hypernymsuite-LEDS": "data/hypernymsuite/LEDS/IsA.jsonl",
    "hypernymsuite-EVAL": "data/hypernymsuite/EVAL/IsA.jsonl",
    "hypernymsuite-SHWARTZ": "data/hypernymsuite/SHWARTZ/IsA.jsonl"}

def get_dataset_to_respath(dataset_to_respath, print_flag=False):
    # remote path 
#     dataset_to_respath = {'hypernymsuite-BLESS': 'log/bert-large-uncased/hypernymsuite/BLESS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'lm_diagnostic_extended-singular': 'log/bert-large-uncased/lm_diagnostic_extended/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.LM_DIAGNOSTIC_EXTENDED.csv', 'clsb-singular': 'log/bert-large-uncased/clsb/singular/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.CLSB.csv', 'hypernymsuite-LEDS': 'log/bert-large-uncased/hypernymsuite/LEDS/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-EVAL': 'log/bert-large-uncased/hypernymsuite/EVAL/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv', 'hypernymsuite-SHWARTZ': 'log/bert-large-uncased/hypernymsuite/SHWARTZ/exp_data_results_anchor_type_Coordinate_remove_Y_PUNC_FULL_concate_or_single_max_anchor_num_10_anchor_scorer_probAvg_filter_obj_True_filter_objects_with_input_True_wnp_True_cpt_False.HYPERNYMSUITE.csv'}

    source_dir = 'spartan:~/cogsci/DAP/'
    target_dir = '../../'
    dataset_to_localpath = defaultdict()
    dataset_rename = {
        'hypernymsuite-BLESS': 'BLESS', 'lm_diagnostic_extended-singular': 'DIAG', 'clsb-singular':'CLSB', 'hypernymsuite-LEDS': 'LEDS', 'hypernymsuite-EVAL': 'EVAL', 'hypernymsuite-SHWARTZ': 
        "SHWARTZ"
    }
    for dataset, path in dataset_to_respath.items():
        path = path.replace(".tsv", ".csv")
        source_path = source_dir + path 
        dataset_l1 = dataset.split("-")[0]
        dataset_l2 = dataset.split("-")[1] 
        target_path = target_dir + path
        scp_string = f"!scp {source_path} {target_path}"
        if print_flag:
            print(scp_string)
            print()
#         print(target_path)
        dataset_to_localpath[dataset_rename[dataset]] = target_path 
#     print(dataset_to_localpath)
    return dataset_to_localpath
dataset_to_localpath = get_dataset_to_respath(dataset_to_respath)
dataset_to_localpath


defaultdict(None,
            {'BLESS': '../../data/hypernymsuite/BLESS/IsA.jsonl',
             'DIAG': '../../data/lm_diagnostic_extended/singular/IsA.jsonl',
             'CLSB': '../../data/clsb/singular/IsA.jsonl',
             'LEDS': '../../data/hypernymsuite/LEDS/IsA.jsonl',
             'EVAL': '../../data/hypernymsuite/EVAL/IsA.jsonl',
             'SHWARTZ': '../../data/hypernymsuite/SHWARTZ/IsA.jsonl'})

In [21]:
def load_data(filepath):
    with open(filepath, 'r', encoding='utf-8') as fin:
        data = fin.readlines()
        data = [eval(x) for x in data]
        new_example = []
        for example in data:
            example['obj_label'] = example['obj_label'][0]
            if example['obj_label'] not in bert_vocab: continue 
            new_example.append(example)
        df = pd.DataFrame(new_example)

        #df['obj_label'] = df['obj_label'].apply(lambda x: eval(x)[0] if isinstance(x, str) else x[0])
    return df 

df = load_data(dataset_to_localpath['DIAG'])
df.head()

,masked_sentences,obj_label,sub_label,sub_label_pl,relation,sub_sister,uuid
0,[A graver is a [MASK].],tool,graver,gravers,IsA,"[bevel, dibber, spreader, hammer, crank, float...",1
1,[A smallmouth is a [MASK].],fish,smallmouth,smallmouths,IsA,[largemouth],2
2,[A pelican is a [MASK].],bird,pelican,pelicans,IsA,"[cormorant, snakebird, tropicbird, darter, anh...",3
3,[A sapsucker is a [MASK].],bird,sapsucker,sapsuckers,IsA,"[wryneck, redhead, piculet, ivorybill, flicker]",4
4,[A mako is a [MASK].],fish,mako,makos,IsA,[porbeagle],5


In [24]:
query_datasets = ['BLESS', "DIAG", "CLSB", "SHWARTZ", "EVAL", "LEDS"]

datasets_to_stats = []
for dataset in query_datasets:
    df = load_data(dataset_to_localpath[dataset])    
    word_pairs = [name for name, group in df.groupby(['sub_label', 'obj_label'])]
    hyper = set(df['obj_label'])
    num_hypon = len(set(df['sub_label']))
    num_hyper = len(hyper)
    num_pairs = len(word_pairs)
    
    #print(f"#Hypo {num_hypon}")
    #print(f"#Hyper {len(hyper)}")
    #print(f"#Hypo-Hyper pairs:",len(word_pairs) )    

    stats = pd.Series({"Dataset": dataset, "#Hypon": num_hypon, "#Hyper": num_hyper, "#Pairs": num_pairs }).T
    datasets_to_stats.append(stats)
    #print("-"*80)
    #print()
datasets_to_stats=pd.concat(datasets_to_stats, axis=1).T
display(datasets_to_stats)
print(datasets_to_stats.to_latex())

,Dataset,#Hypon,#Hyper,#Pairs
0,BLESS,200,85,935
1,DIAG,576,9,576
2,CLSB,508,232,1079
3,SHWARTZ,11061,1101,12724
4,EVAL,621,348,953
5,LEDS,1073,364,1262


\begin{tabular}{lllll}
\toprule
{} &  Dataset & \#Hypon & \#Hyper & \#Pairs \\
\midrule
0 &    BLESS &    200 &     85 &    935 \\
1 &     DIAG &    576 &      9 &    576 \\
2 &     CLSB &    508 &    232 &   1079 \\
3 &  SHWARTZ &  11061 &   1101 &  12724 \\
4 &     EVAL &    621 &    348 &    953 \\
5 &     LEDS &   1073 &    364 &   1262 \\
\bottomrule
\end{tabular}



<ipython-input-24-79d77633e64d>:22: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(datasets_to_stats.to_latex())


# Degbug SHWAERTZ (noisy data)

the sub_label are noisy:
shaadi.com
.hack//sign
risk/reward
dmz//38
s.w.a.t.
f.l.m.
s.r.o.
i.o.u.s.a.
c.r.a.z.y.
brother/sister
m.sc
m.p.g.
m.b.b.s.
d.s.
d.p.o.
n.i.b.
t.n.t.shaadi.com
.hack//sign
risk/reward
dmz//38
s.w.a.t.
f.l.m.
s.r.o.
i.o.u.s.a.
c.r.a.z.y.
brother/sister
m.sc
m.p.g.
m.b.b.s.
d.s.
d.p.o.
n.i.b.
t.n.t.

In [23]:
import os 
import json 
from collections import defaultdict
import pandas as pd
from pathlib import Path
from inflection import pluralize
from util_wordnet import get_sister_terms
import string

def add_plural(word):
    word_plural = pluralize(word)
    return [word, word_plural] if word_plural!=word else [word]

def merge_multiple_labels(df, relations, output_path):
    df= df.query(f"relation in {relations}")
    examples = []
    for name, group in df.groupby(by='sub_label'):
        example = defaultdict()
        example['sub_label'] = name
        example['obj_label'] = [obj[0] for obj in group['obj_label'].values] #.tolist()
        example['masked_sentences'] = [f"{_get_article(name)} {name} is a [MASK].", f"{_get_article(name)} {name} is an [MASK]."]
        examples.append(example)
    examples = pd.DataFrame(examples)
    
    return examples






data_dir = '../data/hypernymysuite/data'
add_plural_ground_truth = False #True 
multi_label = True 
p = Path(data_dir)
paths = list(p.glob(f'*.tsv'))
for path in paths:
    if 'hyperlex_rnd' in path.stem or 'wbless' in path.stem: continue 
    print(path)
    dataset_name = path.stem.upper()
    print(dataset_name)
    if dataset_name !='SHWARTZ': continue 
    out_dir = f"{path.parents[0]}/hypernymsuite/{dataset_name}/"
    Path(out_dir).mkdir(parents=True, exist_ok=True)
    out_path = out_dir + "IsA.jsonl"

    df = pd.read_csv(path, sep='\t').query("label==True")
    if 'relation' in df.columns:
        df = df.query("relation=='hyper'")
    df = df.drop_duplicates().reset_index(drop=True)
    
    
    
    df['masked_sentences'] = df[['word1', 'word2']].apply(lambda x: [_get_article(x[0]) + f" {x[0]} is a [MASK].", _get_article(x[0]) + f" {x[0]} is an [MASK]."], axis=1)
    
    word_pairs = [name for name, group in df.groupby(['word1', 'word2'])]
    hyper = set(df['word2'])
    
    df = df.rename(columns={'word1': 'sub_label', 'word2': 'obj_label'})
    print(f"#Hypo {len(set(df['sub_label']))}")
    print(f"#Hyper {len(hyper)}")
    print(f"#Hypo-Hyper pairs:",len(word_pairs) )
    
    df['uuid'] = df.index 
    df['relation'] = 'IsA'
#     df['sub_sister'] = df['sub_label'].apply(lambda x: get_sister_terms(x, distance_to_hypernym=1))
    
    if add_plural_ground_truth: 
        df['obj_label'] = df['obj_label'].apply(lambda x: add_plural(x)) 
    else: 
        df['obj_label'] = df['obj_label'].apply(lambda x: [x]) 
    
    if not multi_label: 
        df = df[['sub_label', 'obj_label', 'relation', 'masked_sentences', 'uuid']]
        display(df.head())
        examples = df.to_dict(orient='records')
    else: 
        examples = merge_multiple_labels(df, ['IsA'],output_path=out_path ).to_dict(orient="records")
#         save_dict_to_json(examples.to_dict(orient="records") , output_path =output_path )
    examples = remove_noisy_examples(examples)
    save_dict_to_json(examples=examples, output_path=out_path)
    print("-"*80)
    print()


# a_string = '!hi. wh?at is the weat[h]er lik?e.'
# new_string = a_string.translate(str.maketrans('', '', string.punctuation))

# df.query("{}")
df_new = pd.DataFrame(examples)
# df = ['sub_label'] 
sub_labels = set(df_new['sub_label'])
for x in sub_labels:
    if '/' in x or '.' in x: 
        print(x)

../data/hypernymysuite/data/eval.tsv
EVAL
../data/hypernymysuite/data/bibless.tsv
BIBLESS
../data/hypernymysuite/data/bless.tsv
BLESS
../data/hypernymysuite/data/leds.tsv
LEDS
../data/hypernymysuite/data/eval.revisit_hypernym.tsv
EVAL.REVISIT_HYPERNYM
../data/hypernymysuite/data/shwartz.tsv
SHWARTZ
#Hypo 11375
#Hyper 1312
#Hypo-Hyper pairs: 13104
{'sub_label': '.hack//sign', 'obj_label': ['anime'], 'masked_sentences': ['A .hack//sign is a [MASK].', 'A .hack//sign is an [MASK].']}
{'sub_label': '.info', 'obj_label': ['magazine'], 'masked_sentences': ['A .info is a [MASK].', 'A .info is an [MASK].']}
{'sub_label': '3oh!3', 'obj_label': ['band'], 'masked_sentences': ['A 3oh!3 is a [MASK].', 'A 3oh!3 is an [MASK].']}
{'sub_label': '7-tease', 'obj_label': ['album'], 'masked_sentences': ['A 7-tease is a [MASK].', 'A 7-tease is an [MASK].']}
{'sub_label': 'a*teens', 'obj_label': ['band'], 'masked_sentences': ['An a*teens is a [MASK].', 'An a*teens is an [MASK].']}
{'sub_label': 'a.w.o.l.', 'o

In [15]:

new_string

'hi what is the weather like'

In [ ]:
import re
l = re.findall(r"[\w']+|[.,!?;]", ".hack//signs")

filepath = '../data/hypernymysuite/data/hypernymsuite/SHWARTZ/IsA.jsonl'
with open(filepath, 'r', encoding='utf-8') as fin:
    data = fin.readlines()
    data = [eval(x) for x in data]
    df = pd.DataFrame(data)
df.head()
    